In [1]:
#https://stackoverflow.com/a/54594071
from IPython.display import HTML

HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Display code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Display code"></form>''')


In [2]:
import warnings
warnings.filterwarnings('ignore')
from lat_epig.interface import makeScrapeInterface, Parseargs                    # loads the user friendly GUI
makeScrapeInterface()

# Floriano

Text(value='', description='Text 1:')

ToggleButtons(description='Operator', options=('and', 'or', 'not'), value='and')

Text(value='', description='Text 2:')

Text(value='', description='EDCS-ID:')

Text(value='', description='Publication:')

...

Button(description='Inscription Genus...', style=ButtonStyle())

SelectMultiple(description='Inscription genus / personal status:', layout=Layout(display='none'), options=(Non…

SelectMultiple(description='and not:', layout=Layout(display='none'), options=(None, 'Augusti/Augustae', 'carm…

Button(description='Get inscriptions!', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))

In [3]:
from lat_epig.map_interface import make_map_interface
make_map_interface()



Button(description='Update Data File List', style=ButtonStyle())

Dropdown(description='Data File', layout=Layout(width='50%'), options=(('2021-07-30-term1_Petra-248.tsv', Posi…

Text(value='', description='Map Title:')

Dropdown(description='Basemap', index=5, options=(('60 BC provinces', 'roman_empire_60_bc_provinces.shp'), ('A…

RadioButtons(description='Basemap<br/>Styling', index=1, options=(('Light Brown', False), ('Multicoloured', Tr…

...

RadioButtons(description='DPI', index=1, options=(72, 300, 600, 1200), value=300)

Checkbox(value=False, description='Inscription IDs with Points')

Checkbox(value=False, description='(PDF Only) Append Inscriptions')

Button(description='Generate New Maps!', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))

# Interactive Map


In [4]:
HTML('''<script>

</script>
<form action="javascript:IPython.notebook.execute_cells_below()"><input type="submit" id="toggleButton" value="Refresh"></form>''')

In [5]:
import geopandas
import matplotlib.pyplot as plt
from frictionless import extract
from pathlib import Path
from pprint import pprint
import pandas
import os
import subprocess
import geoplot
import matplotlib
import shutil
import glob
# https://geopandas.org/reference/geopandas.GeoDataFrame.html#geopandas.GeoDataFrame
from shapely.geometry import Point
# https://geopandas.org/gallery/plotting_basemap_background.html#sphx-glr-gallery-plotting-basemap-background-py
import contextily as ctx


DATA_DIR = "output"
SUPPORTING_DATA = Path("awmc.unc.edu")
SUPPORTING_DATA = SUPPORTING_DATA / "awmc" / "map_data" / "shapefiles"
ROMAN_ROADS_SHP = SUPPORTING_DATA / "ba_roads" / "ba_roads.shp"
PROVINCES_SHP   = SUPPORTING_DATA / "cultural_data" / "political_shading" / "roman_empire_ad_117" / "shape" / "roman_empire_ad_117.shp"
CITIES_DATA     = Path("cities") / "Hanson2016_Cities_OxREP.csv"

cities_rows = extract(CITIES_DATA)
cities_dataframe = pandas.DataFrame(cities_rows)


#https://cmdlinetips.com/2018/02/how-to-subset-pandas-dataframe-based-on-values-of-a-column/

roads_4326 = geopandas.read_file(ROMAN_ROADS_SHP).to_crs(epsg=4326)
provinces_4326 = geopandas.read_file(PROVINCES_SHP).to_crs(epsg=4326)


cities_geodataframe_4326 = geopandas.GeoDataFrame(
cities_dataframe,
geometry=geopandas.points_from_xy(
  cities_dataframe["Longitude (X)"],
  cities_dataframe["Latitude (Y)"]),
crs="EPSG:4326").to_crs(epsg=4326)
cities_geodataframe_4326.drop("Longitude (X)", inplace=True, axis=1)
cities_geodataframe_4326.drop("Latitude (Y)", inplace=True, axis=1)

#pprint(cities_geodataframe_4326)
geopandas.options.use_pygeos = True